### LSE Data Analytics Online Career Accelerator

# DA201: Data Analytics using Python

# Practical activity: Time-series forecasting

**This is the solution to the practical activity**

A real estate developer, Derek, is planning to start a new project to build family homes. He has access to historic data for the region in which he has acquired land for the project. The historic data includes the prices for houses sold in the area over a 10-year period. The data also includes information about the number of bedrooms in each house ranging from one to five rooms. Derek has asked you to analyse the data for any trends that might be evident. He wants to know the optimal number of bedrooms to maximise the profit for the project.

In this activity, we will be working with a time-series moving average to forecast the prices of houses by:

- plotting the sub-data sets
- resampling to remove noise
- using a time-series moving average.

## Prepare your workstation

In [ ]:
# Import necessary libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## Get multiple outputs in the same cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
# Simple hacks to make plots look better: 

# # Colour pallete to make charts look better.
blue, = sns.color_palette("muted", 1) 

# Darkgrid, white grid, dark, white and ticks.
sns.set_style('whitegrid') 

# Font size of the axes title.
plt.rc('axes', titlesize=18) 

# Font size of the x and y labels.
plt.rc('axes', labelsize=14)    

# Font size of the tick labels.
plt.rc('xtick', labelsize=13,color='#4f4e4e') 

# Font size of the tick labels.
plt.rc('ytick', labelsize=13,color='#4f4e4e')  

# Legend font size.
plt.rc('legend', fontsize=13)

# Controls default text sizes.
plt.rc('font', size=13)          

In [ ]:
# Import CSV file with Pandas.
data = pd.read_csv('raw_sales.csv', index_col=['datesold'], parse_dates=['datesold'])

# View DataFrame.
print(data.shape)
data.head()

## Get to know the data

In [ ]:
# Plot house prices as time-series.
data.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title("Housing Prices")
plt.show(block=False);

# Check for missing values.
data.isna().sum()

## Checking the dataset for the count of houses based on their bedrooms

In [ ]:
# Count the number of values in a specified column of the DataFrame.
print(data['bedrooms'].value_counts())

# Create a plot.
plt.title("Count of number of bedrooms")

sns.despine(left=True);
sns.countplot(x='bedrooms', data=data)

## Creating different datasets based on number of bedrooms

In [ ]:
# Create a copy of the original data for convinience. 
data_sub = data.copy()


# Data set cosnsisting of houses with 1 bedroom: 
df_1 = data_sub[data_sub['bedrooms']==1]
print(df_1.shape)


# Data set cosnsisting of houses with 2 bedrooms: 
df_2 = data_sub[data_sub['bedrooms']==2]
print(df_2.shape)


# Data set cosnsisting of houses with 3 bedrooms: 
df_3 = data_sub[data_sub['bedrooms']==3]
print(df_3.shape)


# Data set cosnsisting of houses with 4 bedrooms: 
df_4 = data_sub[data_sub['bedrooms']==4]
print(df_4.shape)


# Data set cosnsisting of houses with 5 bedrooms: 
df_5 = data_sub[data_sub['bedrooms']==5]
print(df_5.shape)

## Detect outliers

In [ ]:
# Set plot size.
fig, axes = plt.subplots(nrows=3, ncols=2,figsize=(20,20))

# 1 bedroom:
axes[0][0].hist(df_1['price'])
axes[0][0].title.set_text("1 bedroom price histogram")

# 2 bedroom:
axes[0][1].hist(df_2['price'])
axes[0][1].title.set_text("2 bedroom price histogram")

# 3 bedroom:
axes[1][0].hist(df_3['price'])
axes[1][0].title.set_text("3 bedroom price histogram")

# 4 bedroom:
axes[1][1].hist(df_4['price'])
axes[1][1].title.set_text("4 bedroom price histogram")

# 5 bedroom:
axes[2][0].hist(df_5['price'])
axes[2][0].title.set_text("5 bedroom price histogram")


fig.delaxes(axes[2][1])

plt.show()

In [ ]:
# Create a boxplot for 1 bedroom.
# Set figure size.
fig = plt.figure(figsize=(18, 4))

# Create boxplot.
ax = sns.boxplot(x=df_1['price'], whis=1.5)

## Remove outliers

In [ ]:
# The columns you want to search for outliers in.
cols = ['price'] 

# Calculate quantiles and IQR.
# Same as np.percentile but maps (0,1) and not (0,100).
Q1 = df_1[cols].quantile(0.25) 
Q3 = df_1[cols].quantile(0.75)
IQR = Q3 - Q1
IQR

# Return a boolean array of the rows with (any) non-outlier column values.
condition = ~((df_1[cols] < (Q1 - 1.5 * IQR)) | (df_1[cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# Filter our dataframe based on condition.
df_1_non_outlier = df_1[condition]
df_1_non_outlier.shape

In [ ]:
# Plot to see if outliers have been removed: 
# whis = multiplicative factor
fig = plt.subplots(figsize=(12, 2))

ax = sns.boxplot(x=df_1_non_outlier['price'],whis=1.5)

## Additional code

### Data set with 2 bedrooms (`df_2`)

In [ ]:
# whis = multiplicative factor
fig = plt.subplots(figsize=(12, 2))
ax = sns.boxplot(x=df_2['price'], whis=1.5);

In [ ]:
# Histogram plot.
fig = df_2.price.hist(figsize = (12, 4))

In [ ]:
# Outlier removal from houses with 2 bedrooms.
# The columns you want to search for outliers.
cols = ['price']

# Calculate quantiles and IQR.
# Same as np.percentile but maps (0,1) and not (0,100).
Q1 = df_2[cols].quantile(0.25)
Q3 = df_2[cols].quantile(0.75)
IQR = Q3 - Q1
IQR

# Return a boolean array of the rows with (any) non-outlier column values.
condition = ~((df_2[cols] < (Q1 - 1.5 * IQR)) | (df_2[cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# Filter our dataframe based on condition.
df_2_non_outlier = df_2[condition]
df_2_non_outlier.shape

In [ ]:
# Plot to see if outliers have been removed: 
# whis = multiplicative factor
fig = plt.subplots(figsize=(12, 2))

ax = sns.boxplot(x=df_2_non_outlier['price'],whis=1.5);

### Data set with 3 bedrooms (`df_3`)

In [ ]:
# whis = multiplicative factor
fig = plt.subplots(figsize=(12, 2))

ax = sns.boxplot(x=df_3['price'], whis=1.5);

In [ ]:
# Histogram plot.
fig = df_3.price.hist(figsize = (12, 4))

In [ ]:
# Removing outlier from dataset with 3 bedrooms: 
# The columns you want to search for outliers.
cols = ['price'] 

# Calculate quantiles and IQR.
# Same as np.percentile but maps (0,1) and not (0,100).
Q1 = df_3[cols].quantile(0.25)
Q3 = df_3[cols].quantile(0.75)
IQR = Q3 - Q1
IQR

# Return a boolean array of the rows with (any) non-outlier column values.
condition = ~((df_3[cols] < (Q1 - 1.5 * IQR)) | (df_3[cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# Filter our dataframe based on condition.
df_3_non_outlier = df_3[condition]
df_3_non_outlier.shape

In [ ]:
# Plot to see if outliers have been removed: 
# whis = multiplicative factor.
fig = plt.subplots(figsize=(12, 2))

ax = sns.boxplot(x=df_3_non_outlier['price'],whis=1.5)

### Data set with 4 bedrooms (`df_4`)

In [ ]:
# whis = multiplicative factor
import seaborn as sns
fig = plt.subplots(figsize=(12, 2))

ax = sns.boxplot(x=df_4['price'],whis=1.5);

In [ ]:
# Histogram plot.
fig = df_4.price.hist(figsize = (12, 4))

In [ ]:
# Removing outlier from dataset with 4 bedrooms.
# The columns you want to search for outliers.
cols = ['price'] 

# Calculate quantiles and IQR.
# Same as np.percentile but maps (0,1) and not (0,100).
Q1 = df_4[cols].quantile(0.25) 
Q3 = df_4[cols].quantile(0.75)
IQR = Q3 - Q1
IQR

# Return a boolean array of the rows with (any) non-outlier column values.
condition = ~((df_4[cols] < (Q1 - 1.5 * IQR)) | (df_4[cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# Filter our dataframe based on condition.
df_4_non_outlier = df_4[condition]
df_4_non_outlier.shape

In [ ]:
# Plot to see if outliers have been removed: 
# whis = multiplicative factor
fig = plt.subplots(figsize=(12, 2))

ax = sns.boxplot(x=df_4_non_outlier['price'], whis=1.5);

### Data set with 5 bedrooms (`df_5`)

In [ ]:
# whis = multiplicative factor
fig = plt.subplots(figsize=(12, 2))

ax = sns.boxplot(x=df_5['price'], whis=1.5)

In [ ]:
# Histogram plot.
fig = df_5.price.hist(figsize = (12, 4))

In [ ]:
# Removing outlier from dataset with 5 bedrooms: 
# The columns you want to search for outliers.
cols = ['price'] 

# Calculate quantiles and IQR.
# Same as np.percentile but maps (0,1) and not (0,100).
Q1 = df_5[cols].quantile(0.25) 
Q3 = df_5[cols].quantile(0.75)
IQR = Q3 - Q1
IQR

# # Return a boolean array of the rows with (any) non-outlier column values.
condition = ~((df_5[cols] < (Q1 - 1.5 * IQR)) | (df_5[cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# Filter our dataframe based on condition.
df_5_non_outlier = df_5[condition]
df_5_non_outlier.shape

In [ ]:
# Plot to see if outliers have been removed: 
fig = plt.subplots(figsize=(12, 2))

ax = sns.boxplot(x=df_5_non_outlier['price'], whis=1.5)

# 

# 

# Time-series forecasting
## Plot the subset DataFrames

In [ ]:
# Create a plot for 1 bedroom.
# Calculate max and min.
df_1_non_outlier['price'].min()
df_1_non_outlier['price'].max()


# Plotting the time series data.
df_1_non_outlier.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title("Time-series plot for house with 1 bedroom")
plt.show(block=False)

In [ ]:
# Create a plot for 2 bedrooms.
# Calculate max and min.
df_2_non_outlier['price'].min()
df_2_non_outlier['price'].max()


# Plotting the time series data.
df_2_non_outlier.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title("Time-series plot for house with 1 bedroom")
plt.show(block=False)

In [ ]:
# Create a plot for 3 bedrooms.
# Calculate max and min.
df_3_non_outlier['price'].min()
df_3_non_outlier['price'].max()


# Plotting the time series data.
df_3_non_outlier.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title("Time-series plot for house with 1 bedroom")
plt.show(block=False)

In [ ]:
# Create a plot for 4 bedrooms.
# Calculate max and min.
df_4_non_outlier['price'].min()
df_4_non_outlier['price'].max()


# Plotting the time series data.
df_4_non_outlier.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title("Time-series plot for house with 1 bedroom")
plt.show(block=False)

In [ ]:
# Create a plot for 5 bedrooms.
# Calculate max and min.
df_5_non_outlier['price'].min()
df_5_non_outlier['price'].max()


# Plotting the time series data.
df_5_non_outlier.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title("Time-series plot for house with 1 bedroom")
plt.show(block=False)

## Resample the sub-data sets to remove noise

In [ ]:
# 1 bedroom:
# Resampling dataset with 1 bedroom:
df_1_res = df_1_non_outlier.resample('M').mean()
df_1_res.head()


# Dropping the missing values: 
df_1_res.dropna(inplace= True)
df_1_res.isna().sum()


# Plotting the time series data.
df_1_res.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title("Time-series plot after resampling")
plt.show(block=False)

In [ ]:
# 2 bedrooms:
# Resampling dataset with 2 bedrooms.
df_2_res = df_2_non_outlier.resample('M').mean()
df_2_res.head()


# Dropping the missing values: 
df_2_res.dropna(inplace= True)
df_2_res.isna().sum()


# Plotting the time series data.
df_2_res.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title("Time-series plot after resampling")
plt.show(block=False)

In [ ]:
# 3 bedrooms:
# Resampling dataset with 1 bedroom.
df_3_res = df_3_non_outlier.resample('M').mean()
df_3_res.head()

# Dropping the missing values: 
df_3_res.dropna(inplace= True)
df_3_res.isna().sum()

# Plotting the time series data.
df_3_res.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title("Time-series plot after resampling")
plt.show(block=False)

In [ ]:
# 4 bedrooms:
# Resampling dataset with 4 bedrooms.
df_4_res = df_4_non_outlier.resample('M').mean()
df_4_res.head()

# Dropping the missing values: 
df_4_res.dropna(inplace= True)
df_4_res.isna().sum()

# Plotting the time series data.
df_4_res.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title("Time-series plot after resampling")
plt.show(block=False)

In [ ]:
# 5 bedrooms:
# Resampling dataset with 5 bedrooms.
df_5_res = df_5_non_outlier.resample('M').mean()
df_5_res.head()

# Dropping the missing values: 
df_5_res.dropna(inplace= True)
df_5_res.isna().sum()

# Plotting the time series data.
df_5_res.plot(figsize=(12, 4))
plt.legend(loc='best')
plt.title("Time-series plot after resampling")
plt.show(block=False)

## Calculate and plot the simple moving average

In [ ]:
# As discussed in tutorial video:
# This is a function to calculate and plot the simple moving average: 
def plot_moving_average(series, window, plot_intervals=False, scale=1.96):

    rolling_mean = series.rolling(window=window).mean()
    
    plt.figure(figsize=(12,4))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, 'g', label="Simple moving average trend")
    
    # Plot confidence intervals for smoothed values.
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bound = rolling_mean - (mae + scale * deviation)
        upper_bound = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bound, 'r--', label="Upper bound / Lower bound")
        plt.plot(lower_bound, 'r--')
            
    plt.plot(series[window:], label="Actual values")
    plt.legend(loc='best')
    plt.grid(True)

In [ ]:
# 1 bedroom:
# 5 days:
plot_moving_average(df_1_res.price, 5, plot_intervals=True)

# 30-days smoothing:
plot_moving_average(df_1_res.price, 30, plot_intervals=True)

# 90-days smoothing:
plot_moving_average(df_1_res.price, 90, plot_intervals=True)

In [ ]:
# 2 bedrooms:
# 5 days:
plot_moving_average(df_2_res.price, 5, plot_intervals=True)

# 30-days smoothing:
plot_moving_average(df_2_res.price, 30, plot_intervals=True)

# 90-days smoothing:
plot_moving_average(df_2_res.price, 90, plot_intervals=True)

In [ ]:
# 3 bedrooms:
# 5 days:
plot_moving_average(df_3_res.price, 5, plot_intervals=True)

# 30-days smoothing:
plot_moving_average(df_3_res.price, 30, plot_intervals=True)

# 90-days smoothing:
plot_moving_average(df_3_res.price, 90, plot_intervals=True)

In [ ]:
# 4 bedrooms:
# 5 days:
plot_moving_average(df_4_res.price, 5, plot_intervals=True)

# 30-days smoothing:
plot_moving_average(df_4_res.price, 30, plot_intervals=True)

# 90-days smoothing:
plot_moving_average(df_4_res.price, 90, plot_intervals=True)

In [ ]:
# 5 bedrooms:
# 5 days:
plot_moving_average(df_5_res.price, 5, plot_intervals=True)

# 30-days smoothing:
plot_moving_average(df_5_res.price, 30, plot_intervals=True)

# 90-days smoothing:
plot_moving_average(df_5_res.price, 90, plot_intervals=True)